In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
import sklearn as sk
import string
import itertools
from collections import Counter
from gensim.models import Word2Vec, KeyedVectors
import pickle
from sklearn.model_selection import KFold, train_test_split
from sklearn import preprocessing, metrics
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Embedding, Activation,Input,concatenate,Flatten
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model, Model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import tensorflow as tf
from matplotlib import pyplot
from sklearn.metrics import multilabel_confusion_matrix

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
tokenized_full = pd.read_csv("drive/MyDrive/BD4H Project Folder/Tokenized.csv",index_col=0)

In [4]:
tokenized = tokenized_full[:10000]
tokenized = tokenized.reset_index(drop=True)
tokenized

,HADM_ID,TEXT,DESCRIPTION,SUBJECT_ID,ICD9_CODE,ICD9_CODE_primary
0,121936.0,"['patient', 'test', 'information', 'indication...",Report,28063,"['42843', '41071', '5990', '4275', '5849', '50...","['428', '410', '599', '427', '584', '507', '42..."
1,121936.0,"['patient', 'test', 'information', 'indication...",Report,28063,"['42843', '41071', '5990', '4275', '5849', '50...","['428', '410', '599', '427', '584', '507', '42..."
2,121936.0,"['sinus', 'rhythm', 'frequent', 'atrial', 'pre...",Report,28063,"['42843', '41071', '5990', '4275', '5849', '50...","['428', '410', '599', '427', '584', '507', '42..."
3,121936.0,"['rhythm', 'is', 'most', 'likely', 'sinus', 'r...",Report,28063,"['42843', '41071', '5990', '4275', '5849', '50...","['428', '410', '599', '427', '584', '507', '42..."
4,121936.0,"['atrial', 'fibrillation', 'intraventricular',...",Report,28063,"['42843', '41071', '5990', '4275', '5849', '50...","['428', '410', '599', '427', '584', '507', '42..."
...,...,...,...,...,...,...
9995,152740.0,"['resp', 'care', 'received', 'from', 'or', 'an...",Report,31043,"['4414', '5570', '78551', '5849', '5770', '682...","['441', '557', '785', '584', '577', '682', '72..."
9996,152740.0,"['or', 'today', 'for', 'debridement', 'of', 'l...",Report,31043,"['4414', '5570', '78551', '5849', '5770', '682...","['441', '557', '785', '584', '577', '682', '72..."
9997,152740.0,"['npn', 'dd', 'dd', 'dd', 'dd', 'please', 'als...",Report,31043,"['4414', '5570', '78551', '5849', '5770', '682...","['441', '557', '785', '584', '577', '682', '72..."
9998,152740.0,"['respiratory', 'care', 'patient', 'switched',...",Report,31043,"['4414', '5570', '78551', '5849', '5770', '682...","['441', '557', '785', '584', '577', '682', '72..."


In [5]:
tokenized["TEXT"] = tokenized["TEXT"].apply(literal_eval)
tokenized["ICD9_CODE"] = tokenized["ICD9_CODE"].apply(literal_eval)
tokenized

,HADM_ID,TEXT,DESCRIPTION,SUBJECT_ID,ICD9_CODE,ICD9_CODE_primary
0,121936.0,"[patient, test, information, indication, aorti...",Report,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","['428', '410', '599', '427', '584', '507', '42..."
1,121936.0,"[patient, test, information, indication, aorti...",Report,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","['428', '410', '599', '427', '584', '507', '42..."
2,121936.0,"[sinus, rhythm, frequent, atrial, premature, b...",Report,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","['428', '410', '599', '427', '584', '507', '42..."
3,121936.0,"[rhythm, is, most, likely, sinus, rhythm, with...",Report,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","['428', '410', '599', '427', '584', '507', '42..."
4,121936.0,"[atrial, fibrillation, intraventricular, condu...",Report,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","['428', '410', '599', '427', '584', '507', '42..."
...,...,...,...,...,...,...
9995,152740.0,"[resp, care, received, from, or, and, placed, ...",Report,31043,"[4414, 5570, 78551, 5849, 5770, 6822, 72973, 9...","['441', '557', '785', '584', '577', '682', '72..."
9996,152740.0,"[or, today, for, debridement, of, left, flank,...",Report,31043,"[4414, 5570, 78551, 5849, 5770, 6822, 72973, 9...","['441', '557', '785', '584', '577', '682', '72..."
9997,152740.0,"[npn, dd, dd, dd, dd, please, also, see, carev...",Report,31043,"[4414, 5570, 78551, 5849, 5770, 6822, 72973, 9...","['441', '557', '785', '584', '577', '682', '72..."
9998,152740.0,"[respiratory, care, patient, switched, to, cpa...",Report,31043,"[4414, 5570, 78551, 5849, 5770, 6822, 72973, 9...","['441', '557', '785', '584', '577', '682', '72..."


In [6]:
def convert_icd9(x):
    converted_code = []
    for code in x:
        icd9_str = str(code)
        if icd9_str.startswith("E"):
            converted = icd9_str[:4]
        else:
            converted = icd9_str[:3]
        converted_code.append(converted)
    return converted_code
tokenized["ICD9_CODE_primary"] = tokenized["ICD9_CODE"].apply(lambda x: convert_icd9(x))

In [7]:
mlb = preprocessing.MultiLabelBinarizer()
y = mlb.fit_transform(tokenized["ICD9_CODE_primary"])
y = pd.DataFrame(y)
print(y.shape) #full: (399623, 808)

(10000, 363)


In [22]:
mlb2 = preprocessing.MultiLabelBinarizer()
y_regular = mlb2.fit_transform(tokenized["ICD9_CODE"])
y_regular = pd.DataFrame(y_regular)
print(y_regular.shape) #full: (399623, 808)

(10000, 936)


In [7]:
#metrics.precision_score(y_test, yhat_cnn, average='weighted')
#metrics.recall_score(y_test, yhat_cnn, average='weighted')
#metrics.f1_score(y_test, yhat_cnn, average='weighted')

In [8]:
def eval(y_true, y_pred):
    precision_list = []
    recall_list = []
    f1_list = []
    mcm = multilabel_confusion_matrix(y_true, y_pred)
    for i in range(len(mcm)):
        tn, fp, fn, tp = mcm[i].ravel()
        if tp == 0:
            precision = 0
            recall = 0
            f1 = 0
        else:
            precision = tp/ (tp+fp)
            recall = tp/(fn)
            f1 = 2*precision*recall/(precision+recall)
        precision_list.append(precision) 
        recall_list.append(recall)
        f1_list.append(f1)
    return np.mean(precision_list),np.mean(recall_list),np.mean(f1_list)

In [9]:
def label_based_macro_precision(y_true, y_pred):
	
	# axis = 0 computes true positive along columns i.e labels
	l_prec_num = np.sum(np.logical_and(y_true, y_pred), axis = 0)

	# axis = computes true_positive + false positive along columns i.e labels
	l_prec_den = np.sum(y_pred, axis = 0)

	# compute precision per class/label
	l_prec_per_class = l_prec_num/l_prec_den

	# macro precision = average of precsion across labels. 
	l_prec = np.mean(l_prec_per_class)
	return l_prec


In [10]:
def label_based_macro_recall(y_true, y_pred):
    
    # compute true positive along axis = 0 i.e labels
    #l_recall_num = np.sum(np.logical_and(y_true, y_pred), axis = 0)

    # compute true positive + false negatives along axis = 0 i.e columns
    #l_recall_den = np.sum(np.logical_and(y_true, y_pred), axis = 0)
    #l_recall_den = np.sum(y_true, axis = 0)

    # compute recall per class/label
    #l_recall_per_class = l_recall_num/l_recall_den

    tp = np.sum(np.logical_and(y_true, y_pred))
    
    # compute false negatives (Missed Labels) across training examples and labels
    fn = np.sum(np.logical_and(y_true, np.logical_not(y_pred)))
    
    fp = np.sum(np.logical_and(np.logical_not(y_true), y_pred))

    # compute macro averaged recall i.e recall averaged across labels. 
    #l_recall = np.mean(l_recall_per_class)
    
    return np.mean(tp/(tp+fn))

In [11]:
def f1_evaluation_score(y_true, y_pred):
   
    # compute true positives across training examples and labels
    tp = np.sum(np.logical_and(y_true, y_pred))
    
    # compute false negatives (Missed Labels) across training examples and labels
    fn = np.sum(np.logical_and(y_true, np.logical_not(y_pred)))
    
    fp = np.sum(np.logical_and(np.logical_not(y_true), y_pred))

    # compute False Positive across training examples and labels.
    f1 = 2*tp / (2* tp + fp + fn)
    
    return np.mean(f1)

In [12]:
def f1_score(precision,recall):
    return 2*precision*recall/(precision+recall)

In [13]:
def build_w2v_model(x_train):
    #w2v_model = Word2Vec.load("drive/MyDrive/BD4H Project Folder/word2vec.model")
    w2v_model = Word2Vec(x_train, size=300)
    #w2v_model.save("word2vec_test.model")
    #print(w2v_model)
    return w2v_model

In [14]:
def get_sequence(w2v_model,x_train,x_val,x_test):
    num_unique_tokens = len(w2v_model.wv.vocab)
    token = Tokenizer(num_unique_tokens)
    token.fit_on_texts(x_train)
    train_text = token.texts_to_sequences(x_train)
    x_train_seq = pad_sequences(train_text)
    print(x_train_seq.shape) 

    sequences_val = token.texts_to_sequences(x_val)
    x_val_seq = pad_sequences(sequences_val, maxlen=x_train_seq.shape[1]) 
    print(x_val_seq.shape)

    sequences_test = token.texts_to_sequences(x_test)
    x_test_seq = pad_sequences(sequences_test, maxlen=x_train_seq.shape[1]) 
    print(x_test_seq.shape)

    return x_train_seq,x_val_seq,x_test_seq

In [15]:
def build_embedding_vector(w2v_model):

    num_unique_tokens = len(w2v_model.wv.vocab)
    token = Tokenizer(num_unique_tokens)
    embeddings_index = {}
    for w in w2v_model.wv.vocab.keys():
        embeddings_index[w] = w2v_model.wv[w]

    num_words = num_unique_tokens
    embedding_matrix = np.zeros((num_words, 300))
    for word, i in token.word_index.items():
        if i >= num_words:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [16]:
def CNN_base(x_train_seq,y_train,x_val_seq,y_val,x_test_seq,y_test,num_unique_tokens):
    report_input = Input(shape=(x_train_seq.shape[1],), dtype='int32')

    report_encoder = Embedding(num_unique_tokens, 300, weights=[embedding_matrix], input_length=x_train_seq.shape[1], trainable=False)(report_input)
    bigram_branch = Conv1D(filters=250, kernel_size=3, padding='valid', activation='relu', strides=1)(report_encoder)
    bigram_branch = GlobalMaxPooling1D()(bigram_branch)

    #trigram_branch = Conv1D(filters=100, kernel_size=3, padding='valid', activation='relu', strides=1)(tweet_encoder)
    #trigram_branch = GlobalMaxPooling1D()(trigram_branch)
    #fourgram_branch = Conv1D(filters=100, kernel_size=4, padding='valid', activation='relu', strides=1)(tweet_encoder)
    #fourgram_branch = GlobalMaxPooling1D()(fourgram_branch)
    #merged = concatenate([bigram_branch, trigram_branch, fourgram_branch], axis=1)

    merged = Dense(y_train.shape[1])(bigram_branch)
    #merged = Dropout(0.2)(merged)
    #merged = Dense(1)(merged)
    output = Activation('relu')(merged)

    model = Model(inputs=[report_input], outputs=[output])

    #model.compile(optimizer=adam, loss='binary_crossentropy', vmetrics=['accuracy'])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=tf.keras.losses.BinaryCrossentropy(), metrics=[tf.keras.metrics.BinaryAccuracy()])
    #model.summary()

    callbacks = [EarlyStopping(monitor='val_loss')]
    model.fit(x_train_seq, y_train, batch_size=32, epochs=5, validation_data=(x_val_seq, y_val), callbacks = callbacks)
    #model.evaluate(x=x_val_seq, y=y_val)

    yhat_cnn = model.predict(x_test_seq)
    #model.evaluate(x=x_test_seq, y=y_test)
    yhat_cnn = model.predict(x_test_seq)
    yhat_cnn[yhat_cnn>0.5] = 1
    yhat_cnn[yhat_cnn<=0.5] = 0
    yhat_cnn = pd.DataFrame(yhat_cnn,dtype=int)

    precision = label_based_macro_precision(y_test, yhat_cnn) 
    # recall: tp / (tp + fn)
    recall = label_based_macro_recall(y_test, yhat_cnn) 
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_evaluation_score(y_test, yhat_cnn)
    
    return  precision, recall, f1

In [19]:
def CNN_base_test(x_train_seq,y_train,x_val_seq,y_val,x_test_seq,y_test,num_unique_tokens):
    report_input = Input(shape=(x_train_seq.shape[1],), dtype='int32')

    report_encoder = Embedding(num_unique_tokens, 300, input_length=x_train_seq.shape[1], trainable=True)(report_input)
    bigram_branch = Conv1D(filters=250, kernel_size=3, activation='relu',padding='valid',strides=1)(report_encoder)
    bigram_branch = GlobalMaxPooling1D()(bigram_branch)

    merged = Dense(y_train.shape[1])(bigram_branch)

    output = Activation('sigmoid')(merged)
    #trigram_branch = Conv1D(filters=100, kernel_size=3, padding='valid', activation='relu', strides=1)(tweet_encoder)
    #trigram_branch = GlobalMaxPooling1D()(trigram_branch)
    #fourgram_branch = Conv1D(filters=100, kernel_size=4, padding='valid', activation='relu', strides=1)(tweet_encoder)
    #fourgram_branch = GlobalMaxPooling1D()(fourgram_branch)
    #merged = concatenate([bigram_branch, trigram_branch, fourgram_branch], axis=1)

    #merged = Dense(y_train.shape[1])(bigram_branch)
    #merged = Dropout(0.2)(merged)
    #merged = Dense(1)(merged)

    model = Model(inputs=[report_input], outputs=[output])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2), loss=tf.keras.losses.BinaryCrossentropy())
    #model.summary()
    
    callbacks = [EarlyStopping(min_delta=0.0001,patience=2)]
    model.fit(x_train_seq, y_train, batch_size=32, epochs=3, validation_data=(x_val_seq, y_val),callbacks=callbacks)
    #model.evaluate(x=x_val_seq, y=y_val)
    #_, train_acc = model.evaluate(x_train_seq, y_train)
    #_, val_acc = model.evaluate(x_val_seq, y_val)
    #pyplot.plot(history.history['loss'], label='train')
    #pyplot.plot(history.history['val_loss'], label='validation')
    #pyplot.legend()
    #pyplot.show()

    #model.evaluate(x=x_test_seq, y=y_test)
    yhat_cnn = model.predict(x_test_seq)
    yhat_cnn[yhat_cnn>0.5] = 1
    yhat_cnn[yhat_cnn<=0.5] = 0
    yhat_cnn = pd.DataFrame(yhat_cnn,dtype=int)

    precision = label_based_macro_precision(y_test, yhat_cnn) 
    # recall: tp / (tp + fn)
    recall = label_based_macro_recall(y_test, yhat_cnn) 
    # f1: 2 tp / (2 tp + fp + fn)
    #f1 = f1_evaluation_score(y_test, yhat_cnn)
    f1 = f1_score(precision,recall)
    #precision2, recall2, f12 = eval(y_test, yhat_cnn)
    #precision = metrics.precision_score(y_test, yhat_cnn, average=None)
    #recall = metrics.recall_score(y_test, yhat_cnn, average=None)
    #f1 = metrics.f1_score(y_test, yhat_cnn, average=None)
    return  precision, recall, f1

In [20]:
kf = KFold(n_splits = 5, shuffle = True, random_state = 2)
precision_list = []
recall_list = []
f1_list = []

for train_index, test_index in kf.split(tokenized):
    #print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = tokenized["TEXT"].iloc[train_index], tokenized["TEXT"].iloc[test_index]
    y_train,y_test = y.iloc[train_index],y.iloc[test_index]
    x_val = x_train.sample(frac=0.1,random_state=42)
    y_val = y_train.sample(frac=0.1,random_state=42)

    w2v_model = build_w2v_model(x_train)
    num_unique_tokens = len(w2v_model.wv.vocab)
    x_train_seq,x_val_seq,x_test_seq = get_sequence(w2v_model,x_train,x_val,x_test)
    embedding_matrix = build_embedding_vector(w2v_model)

    precision, recall, f1 = CNN_base_test(x_train_seq,y_train,x_val_seq,y_val,x_test_seq,y_test,num_unique_tokens)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)
    
    

print(np.mean(precision_list))
print(np.mean(recall_list))
print(np.mean(f1_list))

(8000, 2174)
(800, 2174)
(2000, 2174)
Epoch 1/3
250/250 [==============================] - 536s 2s/step - loss: 0.1100 - val_loss: 0.0838
Epoch 2/3
250/250 [==============================] - 469s 2s/step - loss: 0.0789 - val_loss: 0.0691
Epoch 3/3
250/250 [==============================] - 461s 2s/step - loss: 0.0699 - val_loss: 0.0651


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy ar

(8000, 2174)
(800, 2174)
(2000, 2174)
Epoch 1/3
250/250 [==============================] - 466s 2s/step - loss: 0.1105 - val_loss: 0.0822
Epoch 2/3
250/250 [==============================] - 465s 2s/step - loss: 0.0787 - val_loss: 0.0698
Epoch 3/3
250/250 [==============================] - 461s 2s/step - loss: 0.0691 - val_loss: 0.0623


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy ar

(8000, 2177)
(800, 2177)
(2000, 2177)
Epoch 1/3
250/250 [==============================] - 463s 2s/step - loss: 0.1110 - val_loss: 0.0802
Epoch 2/3
250/250 [==============================] - 460s 2s/step - loss: 0.0790 - val_loss: 0.0697
Epoch 3/3
250/250 [==============================] - 461s 2s/step - loss: 0.0702 - val_loss: 0.0643


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy ar

(8000, 2177)
(800, 2177)
(2000, 2177)
Epoch 1/3
250/250 [==============================] - 459s 2s/step - loss: 0.1103 - val_loss: 0.0770
Epoch 2/3
250/250 [==============================] - 461s 2s/step - loss: 0.0780 - val_loss: 0.0672
Epoch 3/3
250/250 [==============================] - 461s 2s/step - loss: 0.0691 - val_loss: 0.0625


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy ar

(8000, 2174)
(800, 2174)
(2000, 2174)
Epoch 1/3
250/250 [==============================] - 461s 2s/step - loss: 0.1102 - val_loss: 0.0882
Epoch 2/3
250/250 [==============================] - 457s 2s/step - loss: 0.0788 - val_loss: 0.0755
Epoch 3/3
250/250 [==============================] - 458s 2s/step - loss: 0.0692 - val_loss: 0.0681
0.7826206422020715
0.3767451106278674
0.5078554546630053


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy ar

In [ ]:
kf = KFold(n_splits = 5, shuffle = True, random_state = 2)
precision_list_re = []
recall_list_re = []
f1_list_re = []

for train_index, test_index in kf.split(tokenized):
    #print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = tokenized["TEXT"].iloc[train_index], tokenized["TEXT"].iloc[test_index]
    y_train_regular,y_test_regular = y_regular.iloc[train_index],y_regular.iloc[test_index]
    x_val = x_train.sample(frac=0.1,random_state=42)
    y_val_regular = y_train_regular.sample(frac=0.1,random_state=42)

    w2v_model = build_w2v_model(x_train)
    num_unique_tokens = len(w2v_model.wv.vocab)
    x_train_seq,x_val_seq,x_test_seq = get_sequence(w2v_model,x_train,x_val,x_test)
    embedding_matrix = build_embedding_vector(w2v_model)

    precision_regular, recall_regular, f1_regular = CNN_base_test(x_train_seq,y_train_regular,x_val_seq,y_val_regular,x_test_seq,y_test_regular,num_unique_tokens)
    precision_list_re.append(precision_regular)
    recall_list_re.append(recall_regular)
    f1_list_re.append(f1_regular)
    
    

print(np.mean(precision_list))
print(np.mean(recall_list))
print(np.mean(f1_list))

(8000, 2174)
(800, 2174)
(2000, 2174)
Epoch 1/3
250/250 [==============================] - 465s 2s/step - loss: 0.0698 - val_loss: 0.0482
Epoch 2/3
250/250 [==============================] - 459s 2s/step - loss: 0.0457 - val_loss: 0.0439
Epoch 3/3
250/250 [==============================] - 452s 2s/step - loss: 0.0404 - val_loss: 0.0361


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy ar

(8000, 2174)
(800, 2174)
(2000, 2174)
Epoch 1/3
250/250 [==============================] - 460s 2s/step - loss: 0.0699 - val_loss: 0.0493
Epoch 2/3
128/250 [==============>...............] - ETA: 3:39 - loss: 0.0479

In [ ]:
#https://www.kaggle.com/code/marijakekic/cnn-in-keras-with-pretrained-word2vec-weights/notebook
#https://pianalytix.com/multi-label-text-classification/
#https://towardsdatascience.com/another-twitter-sentiment-analysis-with-python-part-11-cnn-word2vec-41f5e28eda74
#https://medium.datadriveninvestor.com/a-survey-of-evaluation-metrics-for-multilabel-classification-bb16e8cd41cd